##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introducción a las variables

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/variable"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/variable.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/variable.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/variable.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

Se recomienda usar una **variable** de TensorFlow para representar el estado compartido y persistente que su programa manipula. En esta guía se describe cómo crear, actualizar y gestionar instancias de `tf.Variable` en TensorFlow.

Las variables se crean a través de la clase `tf.Variable` y también se les da seguimiento mediante esta misma clase. Una `tf.Variable` representa a un tensor cuyo valor se puede cambiar si se le ejecutan operaciones. Las operaciones (ops) específicas permiten leer y modificar los valores del tensor. Las bibliotecas de alto nivel como `tf.keras` usan `tf.Variable` para almacenar los parámetros del modelo. 

## Preparación

En este artículo se habla sobre la ubicación de las variables. Si desea ver en qué dispositivos están ubicadas sus variables, elimine los comentarios de la línea.

In [ ]:
import tensorflow as tf

# Uncomment to see where your variables get placed (see below)
# tf.debugging.set_log_device_placement(True)

## Creación de una variable

Para crear una variable, proporcione el valor inicial.  La `tf.Variable` tendrá el mismo `dtype` del valor de la inicialización.

In [ ]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

Una variable tiene el mismo aspecto que un tensor y se comporta como tal, además; de hecho, es una estructura de datos respaldada por un `tf.Tensor`. Al igual que con los tensores, tienen un `dtype` y una forma; y pueden exportarse a NumPy.

In [ ]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

La mayoría de las operaciones con tensores funcionan con variables tal como se espera, a pesar de que las variables no se pueden reformar.

In [ ]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.math.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

Tal como se describe arriba, las variables están respaldadas por tensores. El tensor se puede reasignar con `tf.Variable.assign`.  Si se invoca `assign`, por lo común, no se asigna un tensor nuevo; en cambio, se reutiliza la memoria del tensor existente.

In [ ]:
a = tf.Variable([2.0, 3.0])
# This will keep the same dtype, float32
a.assign([1, 2]) 
# Not allowed as it resizes the variable: 
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

Si se usa una variable como un tensor en operaciones, por lo general, se operará en el tensor de respaldo.

Al crear variables nuevas a partir de variables existentes se duplican los tensores de respaldo. Dos variables no compartirán la misma memoria.

In [ ]:
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

## Ciclos de vida, asignación de nombres y observación

En TensorFlow basado en Python, la instancia `tf.Variable` tiene el mismo ciclo de vida que otros objetos de Python. Cuando no hay referencias a una variable, se le quita la asignación automáticamente.

A las variables también se les puede asignar un nombre que puede ser útil para darles seguimiento y depurarlas. Se les puede dar el mismo nombre a dos variables diferentes.

In [ ]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(a == b)

Los nombres de las variables se conservan al guardar y cargar los modelos. Por defecto, las variables de los modelos adquirirán automáticamente nombres únicos de variables, por lo que no será necesario asignárselos uno mismo, a menos que en realidad quiera hacerlo.

A pesar de que las variables son importantes para la diferenciación, algunas no necesitarán ser diferenciadas. Se pueden desactivar los gradientes para una variable, hay que definir `trainable` como falso en el momento de la creación. Un ejemplo de una variable que no necesitará gradientes es un contador de pasos de entrenamiento.

In [ ]:
step_counter = tf.Variable(1, trainable=False)

## Ubicación de variables y tensores

Para un mejor desempeño, TensorFlow intentará ubicar los tensores y las variables en el dispositivo compatible más rápido, y lo hará con su `dtype`. Significa que la mayoría se las variables de ubican en una GPU, en caso de que haya una disponible.

Sin embargo, se puede sobrescribir (<em>override</em>). En este fragmento,  ubique un tensor flotante y una variable en la CPU, incluso aunque haya una GPU disponible. Al activar el registro de la ubicación del dispositivo (consulte la sección [Preparación](#scrollTo=xZoJJ4vdvTrD)), verá dónde se ubica la variable.

Nota: A pesar de que la ubicación manual funciona, las [estrategias de distribución](distributed_training.ipynb) pueden ser las opciones más convenientes y escalables para la optimización del cálculo.

Si este *notebook* se ejecuta en backends diferentes con o sin una GPU, verá un registro (*logging*) diferente. *Tenga en cuenta que la ubicación del dispositivo de registro se debe activar al inicio de la sesión.*

In [ ]:
with tf.device('CPU:0'):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

Se puede establecer la ubicación de una variable o tensor en un dispositivo y hacer el cálculo en otro. Esto agregará una demora, ya que los datos deberán copiarse entre dispositivos.

Sin embargo, probablemente quiera hacerlo de todos modos si tiene varios nodos trabajadores GPU pero solamente quiere una sola copia de las variables.

In [ ]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

Nota: Dado que `tf.config.set_soft_device_placement` se activa por defecto,  aún se ejecutará, incluso aunque ejecute este código en un dispositivo sin GPU.  El paso de multiplicación se producirá en la CPU.

Para más información sobre entrenamiento distribuido, consulte la [guía](distributed_training.ipynb).

## Próximos pasos

Para entender cómo se usan normalmente las variables, consulte nuestra guía sobre [diferenciación automática](autodiff.ipynb).